# Feature Selection
### Overfitting a Decision Tree
This bug was found when Katie was trying to make an overfit decision tree to use as an example in the decision tree mini-project. A decision tree is classically an algorithm that can be easy to overfit; one of the easiest ways to get an overfit decision tree is to use a small training set and lots of features.
If a decision tree is overfit, would you expect the accuracy on a test set to be very high or pretty low?
* Low!

If a decision tree is overfit, would you expect high or low accuracy on the training set?
* High!

### Number of Features and Overfitting
A classic way to overfit an algorithm is by using lots of features and not a lot of training data. You can find the starter code in feature_selection/find_signature.py. Get a decision tree up and training on the training data, and print out the accuracy. How many training points are there, according to the starter code?

How many training points are there, according to the starter code?
* 150

### Accuracy of Your Overfit Decision Tree
What’s the accuracy of the decision tree you just made? (Remember, we're setting up our decision tree to overfit -- ideally, we want to see the test accuracy as relatively low.)

In [ ]:
import pickle
import numpy
numpy.random.seed(42)

### The words (features) and authors (labels), already largely processed.
### These files should have been created from the previous (Lesson 10)
### mini-project.
words_file = "your_word_data.pkl" 
authors_file = "your_email_authors.pkl"
word_data = pickle.load( open(words_file, "rb"))
authors = pickle.load( open(authors_file, "rb") )

### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()

### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]

### your code goes here
from sklearn import tree

# Create classifier
clf = tree.DecisionTreeClassifier(min_samples_split = 40)

# Fit the classifier on the training features and labels
clf.fit(features_train, labels_train)

# Make prediction - Store predictions in a list named pred
pred = clf.predict(features_test)

# Calculate the accuracy on the test data
print("Accuracy: {}".format(clf.score(features_test, labels_test)))

### Identify the Most Powerful Features
Take your (overfit) decision tree and use the feature_importances_ attribute to get a list of the relative importance of all the features being used. We suggest iterating through this list (it’s long, since this is text data) and only printing out the feature importance if it’s above some threshold (say, 0.2--remember, if all words were equally important, each one would give an importance of far less than 0.01). What’s the importance of the most important feature? What is the number of this feature?

In [ ]:
importances = clf.feature_importances_

cnt = 0
for importance in importances:
    if importance > 0.2:
        print("Importance of the most important feature: {}".format(importance))
        print("Number of this feature: {}".format(cnt))
    cnt += 1

print()
# Alternative solution
for index, importance in enumerate(importances):
    if importance > 0.2:
        print("Importance of the most important feature: {}".format(importance)) 
        print("Number of this feature: {}".format(index))

### Use TfIdf to Get the Most Important Word
In order to figure out what words are causing the problem, you need to go back to the TfIdf and use the feature numbers that you obtained in the previous part of the mini-project to get the associated words. You can return a list of all the words in the TfIdf by calling get_feature_names() on it; pull out the word that’s causing most of the discrimination of the decision tree. What is it? Does it make sense as a word that’s uniquely tied to either Chris Germany or Sara Shackleton, a signature of sorts?

In [ ]:
words = vectorizer.get_feature_names()
print("Word: {}".format(words[33614]))

* Even though our training data is limited, we still have a word that is highly indicative of author.

### Remove, Repeat
This word seems like an outlier in a certain sense, so let’s remove it and refit. Go back to text_learning/vectorize_text.py, and remove this word from the emails using the same method you used to remove “sara”, “chris”, etc. Rerun vectorize_text.py, and once that finishes, rerun find_signature.py. Any other outliers pop up? What word is it? Seem like a signature-type word? (Define an outlier as a feature with importance >0.2, as before).

In [ ]:
import re
import string
import os
from nltk.stem.snowball import SnowballStemmer

def parseOutText(f):
    """ given an opened email file f, parse out all text below the
        metadata block at the top
        (in Part 2, you will also add stemming capabilities)
        and return a string that contains all the words
        in the email (space-separated) 
        
        example use case:
        f = open("email_file_name.txt", "r")
        text = parseOutText(f)
        
        """

    f.seek(0)  ### go back to beginning of file (annoying)
    all_text = f.read()

    ### split off metadata
    content = all_text.split("X-FileName:")
    words = ""
    if len(content) > 1:
        ### remove punctuation
        text_string = content[1].translate(str.maketrans("", "", string.punctuation))

        ### project part 2: comment out the line below
        # words = text_string

        ### split the text string into individual words, stem each word,
        ### and append the stemmed word to words (make sure there's a single
        ### space between each stemmed word)
        stemmer = SnowballStemmer("english")
        text_string = text_string.split()
        for text in text_string:
            words = words + stemmer.stem(text) + " "

    return words

from_sara  = open("from_sara.txt", "r")
from_chris = open("from_chris.txt", "r")

from_data = []
word_data = []

### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list
temp_counter = 0

for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        # temp_counter += 1
        if temp_counter < 200:
            path = os.path.join('..', path[:-1])
            email = open(path, "r")

            ### use parseOutText to extract the text from the opened email
            text = parseOutText(email)

            ### use str.replace() to remove any instances of the words
            ### ["sara", "shackleton", "chris", "germani"]
            ### remove sshacklensf
            unwanted = ["sara", "shackleton", "chris", "germani", "sshacklensf"]
            for words in unwanted:
                text = text.replace(words, "")

            ### append the text to word_data
            word_data.append(text)

            ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
            if name == "sara":
                from_data.append(0)
            else:
                from_data.append(1)

            email.close()

print("emails processed")
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_word_data_new.pkl", "wb") )
pickle.dump( from_data, open("your_email_authors_new.pkl", "wb") )

In [ ]:
words_file = "your_word_data_new.pkl" 
authors_file = "your_email_authors_new.pkl"
word_data = pickle.load( open(words_file, "rb"))
authors = pickle.load( open(authors_file, "rb") )

### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()

### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]

# Create classifier
clf = tree.DecisionTreeClassifier(min_samples_split = 40)

# Fit the classifier on the training features and labels
clf.fit(features_train, labels_train)

# Make prediction - Store predictions in a list named pred
pred = clf.predict(features_test)

# Calculate the accuracy on the test data
print("Accuracy: {}".format(clf.score(features_test, labels_test)))

importances = clf.feature_importances_
words = vectorizer.get_feature_names()

for index, importance in enumerate(importances):
    if importance > 0.2:
        print("Importance of the most important feature: {}".format(importance)) 
        print("Number of this feature: {}".format(index))
        print("Word: {}".format(words[index]))

### Checking Important Features Again
Update vectorize_test.py one more time, and rerun. Then run find_signature.py again. Any other important features (importance>0.2) arise? How many? Do any of them look like “signature words”, or are they more “email content” words, that look like they legitimately come from the text of the messages?

In [ ]:
from_sara  = open("from_sara.txt", "r")
from_chris = open("from_chris.txt", "r")

from_data = []
word_data = []

### temp_counter is a way to speed up the development--there are
### thousands of emails from Sara and Chris, so running over all of them
### can take a long time
### temp_counter helps you only look at the first 200 emails in the list
temp_counter = 0

for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
    for path in from_person:
        ### only look at first 200 emails when developing
        ### once everything is working, remove this line to run over full dataset
        # temp_counter += 1
        if temp_counter < 200:
            path = os.path.join('..', path[:-1])
            email = open(path, "r")

            ### use parseOutText to extract the text from the opened email
            text = parseOutText(email)

            ### use str.replace() to remove any instances of the words
            ### ["sara", "shackleton", "chris", "germani"]
            ### remove sshacklensf
            unwanted = ["sara", "shackleton", "chris", "germani", "sshacklensf", "cgermannsf"]
            for words in unwanted:
                text = text.replace(words, "")

            ### append the text to word_data
            word_data.append(text)

            ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
            if name == "sara":
                from_data.append(0)
            else:
                from_data.append(1)

            email.close()

print("emails processed")
from_sara.close()
from_chris.close()

pickle.dump( word_data, open("your_word_data_new2.pkl", "wb") )
pickle.dump( from_data, open("your_email_authors_new2.pkl", "wb") )

In [ ]:
words_file = "your_word_data_new2.pkl" 
authors_file = "your_email_authors_new2.pkl"
word_data = pickle.load( open(words_file, "rb"))
authors = pickle.load( open(authors_file, "rb") )

### test_size is the percentage of events assigned to the test set (the
### remainder go into training)
### feature matrices changed to dense representations for compatibility with
### classifier functions in versions 0.15.2 and earlier
features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()

### a classic way to overfit is to use a small number
### of data points and a large number of features;
### train on only 150 events to put ourselves in this regime
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]

# Create classifier
clf = tree.DecisionTreeClassifier(min_samples_split = 40)

# Fit the classifier on the training features and labels
clf.fit(features_train, labels_train)

# Make prediction - Store predictions in a list named pred
pred = clf.predict(features_test)

# Calculate the accuracy on the test data
print("Accuracy: {}".format(clf.score(features_test, labels_test)))

importances = clf.feature_importances_
words = vectorizer.get_feature_names()

for index, importance in enumerate(importances):
    if importance > 0.2:
        print("Importance of the most important feature: {}".format(importance)) 
        print("Number of this feature: {}".format(index))
        print("Word: {}".format(words[index]))

* There is one more important word ("houectect"). It doesn't look like an obvious signature word so let's keep moving without removing it.

### Accuracy of the Overfit Tree
What’s the accuracy of the decision tree now? We've removed two "signature words", so it will be more difficult for the algorithm to fit to our limited training set without overfitting. Remember, the whole point was to see if we could get the algorithm to overfit--a sensible result is one where the accuracy isn't that great!

* 0.8122866894197952